In [2]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3



# SQLite to Store Financial Analysis on the local drive


<!-- PELICAN_BEGIN_SUMMARY -->

After merging and reshaping the datasets, we can use SQLite to safely storing, organizing and manipulating data in smaller environments.
SQLite is a flexible database can do real work in real business environments. 
This database tool can be useful for accountants to fo financial analysis.


<!-- PELICAN_END_SUMMARY -->

In [3]:
#Name of Excel xlsx file. SQLite database will have the same name and extension .db
filename="data/Premium_Loss" 
con=sqlite3.connect(filename+".db")
wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
#     wb[sheet].to_sql(sheet,con, index=False)
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')
con.commit()
con.close()

C:\apps\python36\python-3.6.3.amd64\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
